In [26]:
import math

def pad(block_size : int , text : str) -> str:
    pad_value = block_size - len(text) % block_size
    return text + chr(pad_value) * pad_value

text = "YELLOW SUBMARINE"
print("Padded:- " , pad(16 , text))

Padded:-  YELLOW SUBMARINE


### CBC decrypt AES

In [21]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
import base64

def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def aes_ecb_decrypt(ct , key):
    cipher = AES.new(key , AES.MODE_ECB)
    return cipher.decrypt(ct)

def aes_cbc_decrypt(ct , key , IV):
    blocks = [ct[i:i+16] for i in range(0, len(ct), 16)]
    plaintext = []
    for i,block in enumerate(blocks):
        ith_block = aes_ecb_decrypt(block , key)
        if i == 0:
            pt = byte_xor(ith_block , IV)
        else:
            pt = byte_xor(ith_block , blocks[i-1])
        plaintext.append(pt)
    return b"".join(plaintext)


file_path = '/run/media/biprarshi/COMMON/files/Crypto/CryptoPals/data/set2q10.txt'
with open(file_path, 'r', ) as f:
    ciphertext = base64.b64decode(f.read())

key = b"YELLOW SUBMARINE"
IV = b'\x00' * 16
print("Plaintext")
print(aes_cbc_decrypt(ciphertext , key, IV).decode('utf-8'))

Plaintext
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make 

In [32]:
from random import randint
from Crypto.Cipher.AES import block_size
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad

BLOCK_SIZE = 16


def byte_xor(a, b):
    return bytes(x ^ y for x, y in zip(a, b))

def aes_ecb_encrypt(data, key):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(data)

def aes_cbc_encrypt(pt, key, iv):
    pt = pad(pt, BLOCK_SIZE)
    cipher = AES.new(key, AES.MODE_ECB)

    blocks = [pt[i:i+BLOCK_SIZE] for i in range(0, len(pt), BLOCK_SIZE)]
    ciphertext = []

    prev = iv
    for block in blocks:
        xored = byte_xor(block, prev)
        encrypted = cipher.encrypt(xored)
        ciphertext.append(encrypted)
        prev = encrypted

    return b''.join(ciphertext)

class AesEncryptOracle:
    @staticmethod
    def random_pad(bin):
        return Random.new().read(randint(5, 10)) + bin + Random.new().read(randint(5, 10))
    @staticmethod
    def encrypt(pt):
        padded_plaintext = AesEncryptOracle.random_pad(pt)
        key = Random.new().read(BLOCK_SIZE)

        if randint(0, 1) == 0:
            return "ECB", aes_ecb_encrypt(
                pad(padded_plaintext, BLOCK_SIZE), key
            )
        else:
            return "CBC", aes_cbc_encrypt(
                padded_plaintext, key, Random.new().read(BLOCK_SIZE)
            )

    @staticmethod
    def count_repetitions(ciphertext):
        blocks = [
            ciphertext[i:i+BLOCK_SIZE]
            for i in range(0, len(ciphertext), BLOCK_SIZE)
        ]
        return len(blocks) - len(set(blocks))

    @staticmethod
    def detect_cipher(ciphertext):
        if AesEncryptOracle.count_repetitions(ciphertext) > 0:
            return "ECB"
        else:
            return "CBC"

oracle = AesEncryptOracle()
input_data = bytes([0]*64)
for _ in range(1000):
        encryption_used, ciphertext = AesEncryptOracle.encrypt(input_data)
        encryption_detected = AesEncryptOracle.detect_cipher(ciphertext)
        assert encryption_used == encryption_detected
print("All tests passed.")

All tests passed.
